In [9]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [10]:
Today = '2024-06-30'

print_to_excel = False

In [11]:
query = f"""
select * 
from xxwh."OPRECConso_detailed"
where "periodDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
and "rowType" != 'on hand'
"""
data_OPRECConso_detailed = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:174: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [14]:
data_OPRECConso_detailed_work = data_OPRECConso_detailed[['businessUnit','customer','appliedPrice','dealPriceDealCurrency','currency','appliedEndDate','quantity','unshippedDealAmount']]
# merge segments
data_OPRECConso_detailed_work['businessSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'businessUnit',merge_col='ocpSegment')
data_OPRECConso_detailed_work['customerCode'] = merge_Mapping(data_OPRECConso_detailed_work,'customer')
data_OPRECConso_detailed_work['customerSegment'] = merge_SalesUnits(data_OPRECConso_detailed_work,'customerCode',merge_col='ocpSegment')
# work with data
data_OPRECConso_detailed_work_not_na = data_OPRECConso_detailed_work[data_OPRECConso_detailed_work.unshippedDealAmount!=0].drop('quantity', axis=1).dropna(subset='appliedPrice').reset_index(drop=True)
data_OPRECConso_detailed_work_not_na['Amount_appliedPrice'] = data_OPRECConso_detailed_work_not_na.appliedPrice*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na['Amount_dealPriceDealCurrency'] = data_OPRECConso_detailed_work_not_na.dealPriceDealCurrency*data_OPRECConso_detailed_work_not_na.unshippedDealAmount
data_OPRECConso_detailed_work_not_na = Period(data_OPRECConso_detailed_work_not_na, Today, 'appliedEndDate')
### to excel
if print_to_excel==True:
    data_print = data_OPRECConso_detailed_work_not_na[['businessSegment','businessUnit','customerSegment','currency','Amount_appliedPrice','Amount_dealPriceDealCurrency','appliedEndDate','Days','Period']]
    Output_file = f'{str(date.today())}_ECH_deals_OPRECConso_detailed.xlsx'
    data_print.to_excel(Output_file, index=False)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['TRADING' 'RUFERT' 'RUFERT' ... 'RUFERT' 'RUFERT' 'RUFERT']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_18300\1837765220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [13]:
mapping_data = data_OPRECConso_detailed_work.loc[data_OPRECConso_detailed_work.customerSegment=='External',['customer','customerCode','customerSegment']].drop_duplicates()
new_list(mapping_data,Output_file,'mapping_sheet')

NameError: name 'Output_file' is not defined

In [15]:
data_OPRECConso_detailed_work_not_na

,businessUnit,customer,appliedPrice,dealPriceDealCurrency,currency,appliedEndDate,unshippedDealAmount,businessSegment,customerCode,customerSegment,Amount_appliedPrice,Amount_dealPriceDealCurrency,Days,Period
0,CHTRD,EUROCHEM ANTWERPEN NV,217.76,217.76,EUR,2024-07-31,340.00,TRADING,BEANF,EURPROD,74038.4,74038.4,31,1M-3M
1,RUNEV,RUEMO,18979.00,18979.00,RUB,2024-04-30,-200.00,RUFERT,RUEMO,RUFERT,-3795800.0,-3795800.0,-61,<1M
2,AECOR,RUEMO,277.00,277.00,USD,2024-07-31,10000.00,TRADING,RUEMO,RUFERT,2770000.0,2770000.0,31,1M-3M
3,RUNEV,RUEMO,12980.00,12980.00,RUB,2024-08-31,-3300.00,RUFERT,RUEMO,RUFERT,-42834000.0,-42834000.0,62,1M-3M
4,RUNEV,RUEMO,11790.00,11790.00,RUB,2024-07-31,-100.10,RUFERT,RUEMO,RUFERT,-1180179.0,-1180179.0,31,1M-3M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20797,BEANF,CHTRD,508.88,508.88,USD,2024-06-30,-250.00,EURPROD,CHTRD,TRADING,-127220.0,-127220.0,0,<1M
20798,RUEMO,"LLC ""PG ""Fosforit""",39890.00,39890.00,RUB,2024-06-30,9057.30,RUFERT,External,External,361295697.0,361295697.0,0,<1M
20799,RUMNM,"OOO ""LYuBERGAZ""",14000.00,14000.00,RUB,2024-07-31,-52.16,RUFERT,External,External,-730240.0,-730240.0,31,1M-3M
20800,RUEMO,"OOO ""LYuBERGAZ""",14000.00,14000.00,RUB,2024-07-31,-52.16,RUFERT,External,External,-730240.0,-730240.0,31,1M-3M


In [16]:
query = f"""
select * 
from "RISKACCESS"."unifiedOcp"
where "reportDate" = TO_DATE('{Today}', 'YYYY-MM-DD')
and "sourceTable" in ('flowDealsReportChanges_inventory','unifiedMaterialMovementSlice_inventory')
"""
data = export_from_RISKCUSTOM(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [27]:
data_work = data[['sourceTable','accountName','holding','buCode','ocpSegment','internalFlag','maturityDateTo','volumeUsd','salesUnitsId']]
# data.loc[:,'balanceItem':]
data_work

,sourceTable,accountName,holding,buCode,ocpSegment,internalFlag,maturityDateTo,volumeUsd,salesUnitsId
0,flowDealsReportChanges_inventory,NK,EUROCHEM,USBFC,RUFERT,External,2024-06-30,-1.222742e+06,USBFC
1,flowDealsReportChanges_inventory,OTHER PRODUCTS,EUROCHEM,BEANF,RUFERT,External,2024-06-30,NaN,RUNEV
2,flowDealsReportChanges_inventory,OTHER PRODUCTS,EUROCHEM,RUNEV,RUFERT,External,2024-06-30,NaN,RUNEV
3,flowDealsReportChanges_inventory,OTHER PRODUCTS,EUROCHEM,RUFSF,RUFERT,External,2024-06-30,NaN,RUNEV
4,flowDealsReportChanges_inventory,OTHER PRODUCTS,EUROCHEM,RUEMO,RUFERT,External,2024-06-30,NaN,RUNEV
...,...,...,...,...,...,...,...,...,...
806,unifiedMaterialMovementSlice_inventory,TSP,EUROCHEM,BRSFT,SAMSALES,External,2024-06-30,2.148862e+06,BRSFT
807,unifiedMaterialMovementSlice_inventory,OTHER PHOSPHATE,EUROCHEM,BRSFT,SAMSALES,External,2024-06-30,3.058307e+08,BRSFT
808,unifiedMaterialMovementSlice_inventory,OTHER CHLOR ALCALI,EUROCHEM,BRSFT,SAMSALES,External,2024-06-30,NaN,BRSFT
809,unifiedMaterialMovementSlice_inventory,MAP,EUROCHEM,BRSFT,SAMSALES,External,2024-06-30,8.781250e+05,BRSFT


In [32]:
appliedEndDate':]
# productGroupNine

SyntaxError: unterminated string literal (detected at line 1) (676762419.py, line 1)